In [1]:
import os
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import xgboost
from xgboost import XGBClassifier
from xgboostextension import XGBRanker
#%matplotlib notebook

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 2000)
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
data_path = '/home/mvkrein/etf_model/data'
etf_data_file = os.path.join(data_path,'etf_new_var_20180910.csv')
etf_data = pd.read_csv(etf_data_file,index_col=0)

In [4]:
etf_study_file = os.path.join(data_path,'ETF_list_min_6yr_history.csv')
etf_list = pd.read_csv(etf_study_file,index_col=0)
etf_data.sort_values(['Date','sym'],ascending=True,inplace=True)
etf_data.reset_index(drop=True,inplace=True)

In [5]:
drop_columns = ['Date','sym','p', 'v', 'p_L05', 'v_L05', 'p_L10', 'v_L10', 'p_L21', 'v_L21', 'p_L42', 'v_L42', 'p_L63', \
                'v_L63', 'p_L84', 'v_L84', 'p_L126', 'v_L126', 'p_L189', 'v_L189', 'p_L252', 'v_L252',\
                'p_L-21', 'v_L-21', 'delta_p_L05', 'delta_p_L10', 'delta_p_L21', 'delta_p_L42', 'delta_p_L63', \
                'delta_p_L84', 'delta_p_L126', 'delta_p_L189', 'delta_p_L252', 'delta_p_L-21', 'delta_v_L05',\
                'delta_v_L10', 'delta_v_L21', 'delta_v_L42', 'delta_v_L63', 'delta_v_L84', 'delta_v_L126', \
                'delta_v_L189', 'delta_v_L252','rank_p_L-21', 'ivv_delta_p_L-21', 'target']

rank_col = ['rank_p_L05','rank_p_L10','rank_p_L21','rank_p_L42',\
            'rank_p_L63','rank_p_L84','rank_p_L126','rank_p_L189','rank_p_L252']

In [6]:
all_columns = list(etf_data.columns)

In [7]:
include_columns = [x for x in all_columns if x not in drop_columns]
# include_columns = ['rank_p_L05','rank_p_L10','rank_p_L21','rank_p_L42',\
#             'rank_p_L63','rank_p_L84','rank_p_L126','rank_p_L189','rank_p_L252']
# include_columns = ['rank_p_L05','rank_p_L21','rank_p_L42','rank_p_L252']

In [8]:
dates = list(etf_data['Date'].unique())
# dates[1671]

In [9]:
#Each year has 252 trading dates.  Need two years to fully develop variables.
dt1 = 504 #This is the first day that all variables are developed
# make all dates relative to dt1
# for one year training - add 252
# to evaluate for one month outside the training window - add 273
# to predict for the first day outside of the evaluation window (have to lag 21) - add 294
dt_end = (len(dates) - 273 - 21) - 1
# dt_end = 505
rpeat = len(etf_data['sym'].unique())
# etf_predict_file = os.path.join(data_path,'etf_pred_xgboost_binary_reg_20180928.csv')
k = 0

In [11]:
for i in range(dt1,dt_end):
    print("Building model to predict for ",dates[i+273])
    x_train = etf_data.loc[((etf_data['Date']>=dates[i]) & (etf_data['Date']<dates[i+231])),include_columns] #train with 12 mos
    y_train = etf_data.loc[((etf_data['Date']>=dates[i]) & (etf_data['Date']<dates[i+231])),['target']] #train with 12 mos
    x_eval = etf_data.loc[((etf_data['Date']>=dates[i+231]) & (etf_data['Date']<dates[i+252])),include_columns] #eval with 1 mos
    y_eval = etf_data.loc[((etf_data['Date']>=dates[i+231]) & (etf_data['Date']<dates[i+252])),['rank_p_L-21']] #eval with 1 mos
#     x_train, x_eval, y_train, y_eval = train_test_split\
#     (etf_data.loc[((etf_data['Date']>=dates[i]) & (etf_data['Date']<dates[i+252])),include_columns],\
#      etf_data.loc[((etf_data['Date']>=dates[i]) & (etf_data['Date']<dates[i+252])),['target']],\
#      test_size=0.2, random_state=54321)
#     x_eval = x_train
#     y_eval = y_train
#predict one day
    x_test = etf_data.loc[(etf_data['Date']==dates[i+273]),include_columns] #predict one day-must be 21 days removed from training
    y_test = etf_data.loc[(etf_data['Date']==dates[i+273]),['target']] #predict if etf >= market
    returns = etf_data.loc[(etf_data['Date']==dates[i+273]),['delta_p_L-21']]
    mkt_return = etf_data.loc[(etf_data['Date']==dates[i+273]),['ivv_delta_p_L-21']]
    x_train_nmpy = x_train.as_matrix()
    y_train_nmpy = y_train.as_matrix()
    x_eval_nmpy = x_eval.as_matrix()
    y_eval_nmpy = y_eval.as_matrix()
    x_test_nmpy = x_test.as_matrix()
    y_test_nmpy = y_test.as_matrix()
    returns_nmpy = returns.as_matrix()
    mkt_return_nmpy = mkt_return.as_matrix()
    x_train_groups = np.repeat(rpeat,231)
    x_eval_groups = np.repeat(rpeat,21)
    x_test_groups = np.repeat(rpeat,1)
    train_xgb = xgboost.DMatrix(x_train_nmpy,label=y_train_nmpy,feature_names=include_columns)
    train_xgb.set_group(x_train_groups)
    eval_xgb = xgboost.DMatrix(x_eval_nmpy,label=y_eval_nmpy,feature_names=include_columns)
    eval_xgb.set_group(x_eval_groups)
    pred_xgb = xgboost.DMatrix(x_test_nmpy,feature_names=include_columns)
    pred_xgb.set_group(x_test_groups)
#turn down learning rate from default value of 0.3, otherwise accept defaults

    param = {'eta':0.3,'seed':54321,'objective':'binary:logistic','max_depth':4, 'subsample': 0.8,'scale_pos_weight':1,\
             'colsample_bytree':0.8,'min_child_weight':1,'gamma':10,'nthread':32,'eval_metric':'auc'\
            ,'lambda':5,'alpha':2}
    param['nthread']=32  #change this to the number of cores on your machine
    param['random_state']=54321 #random number to start
    param['silent']=1 #set to 1 if you don't want all the output

    evallist = [(train_xgb,'train'),(eval_xgb,'eval')]
#     evallist = [(train_xgb,'train')]
    plst = param.items()
    num_round = 500 #max number of rounds -- shouldn't ever get there with early stopping
    evals_result = {}
    bst = xgboost.train(plst,train_xgb,num_round,evals=evallist,early_stopping_rounds=30,verbose_eval=False,\
                        evals_result=evals_result)

    y_model = bst.predict(pred_xgb)
    y_model_array = y_model.reshape((y_test_nmpy.shape))
#     y_model_df = pd.DataFrame(y_model_array,columns=['probability'])
#     y_model_df.sort_values(by='probability',ascending=False,inplace=True)
#     check_val = y_model_df.iloc[4][0]
    y_check = np.column_stack((y_test_nmpy, y_model_array,mkt_return_nmpy,returns_nmpy))
    y_check_df = pd.DataFrame(y_check,columns=['true_value','model_value','mkt_return','returns'])
    y_check_df.sort_values('model_value',inplace=True,ascending=False)
    var_fscore = pd.DataFrame.from_dict(bst.get_fscore(),orient='index')
    var_fscore.columns = ['fscore']
    var_fscore.sort_values(by='fscore',ascending=False,inplace=True)
#     train_error = evals_result['train']['auc'][len(evals_result['train']['auc'])-1]
#     eval_error = evals_result['eval']['auc'][len(evals_result['eval']['auc'])-1]
    number_of_rounds = len(evals_result['train']['auc'])
    train_error = evals_result['train']['auc'][bst.best_iteration]
    eval_error = evals_result['eval']['auc'][bst.best_iteration]
#     number_of_rounds = len(evals_result['eval']['ndcg'])
    print('Model Eval Results:')
    print('Training auc: ',train_error)
    print('Evaluation auc: ',eval_error)
    print('Number of rounds: ',number_of_rounds)
    print('Model Variable Importance:')
    print(var_fscore.iloc[0:10])
    print('*****************************')
    confusion = confusion_matrix(y_test_nmpy, (y_model_array>=0.8).astype(int))
#     fpr, tpr, thresholds = roc_curve(y_test_nmpy, (y_model_array>0.5).astype(int), pos_label=1)
    precision = confusion[1,1]/(confusion[1,1]+confusion[0,1])
    print('xgboost prediction - confusion matrix\n', confusion,'\nprecision: ',round(precision,3))
    print('*****************************')
    print("Model Top 5 Picks")
    print(" Avg rtn top 5:", round(y_check_df['returns'][0:5].mean(),4))
    print(y_check_df.head(5))
    print("Model Bottom 5 Picks")
    print(" Avg rtn bottom 5:", round(y_check_df['returns'][-5:].mean(),4))
    print(y_check_df.tail(5))
    print('********************************************************')
    etf_predict_file = os.path.join(data_path,('etf_pred_xgboost_20180929_'+str(k%21)+'.csv'))
    if k < 21:
        predict_data = etf_data.loc[(etf_data['Date']==dates[i+273]),['Date','sym','target']]
        predict_data['predict'] = y_model_array
        predict_data.to_csv(etf_predict_file)
    else:
        predict_data = etf_data.loc[(etf_data['Date']==dates[i+273]),['Date','sym','target']]
        predict_data['predict'] = y_model_array
        predict_data.to_csv(etf_predict_file,header=False,mode='a')
    k+=1

Building model to predict for  2015-02-05
Model Eval Results:
Training auc:  0.976447
Evaluation auc:  0.560165
Number of rounds:  224
Model Variable Importance:
                      fscore
w252_rank_p_L252_avg      59
w189_rank_p_L252_avg      29
w189_rank_p_L42_avg       29
w42_rank_p_L21_avg        26
w63_rank_p_L63_avg        25
w252_rank_p_L126_avg      22
w252_rank_p_L21_avg       22
w189_rank_p_L21_avg       21
w84_rank_p_L252_avg       20
w252_rank_p_L189_avg      20
*****************************
xgboost prediction - confusion matrix
 [[106   0]
 [ 41   6]] 
precision:  1.0
*****************************
Model Top 5 Picks
 Avg rtn top 5: 0.0411
     true_value  model_value  mkt_return   returns
70          1.0     0.963484    0.010741  0.071770
83          1.0     0.950708    0.010741  0.038518
88          1.0     0.907849    0.010741  0.069374
129         1.0     0.866472    0.010741  0.013101
113         1.0     0.810586    0.010741  0.012604
Model Bottom 5 Picks
 Avg rtn bot

Model Eval Results:
Training auc:  0.966128
Evaluation auc:  0.577536
Number of rounds:  157
Model Variable Importance:
                      fscore
w252_rank_p_L252_avg      47
w126_rank_p_L252_avg      28
w252_rank_p_L126_avg      26
w63_rank_p_L252_avg       21
w189_rank_p_L42_avg       19
w84_rank_p_L21_avg        18
w252_rank_p_L63_avg       18
w189_rank_p_L252_avg      18
w21_rank_p_L252_avg       17
w21_rank_p_L126_avg       16
*****************************
xgboost prediction - confusion matrix
 [[65  3]
 [84  1]] 
precision:  0.25
*****************************
Model Top 5 Picks
 Avg rtn top 5: -0.0161
     true_value  model_value  mkt_return   returns
133         0.0     0.885989   -0.008756 -0.013522
71          0.0     0.866243   -0.008756 -0.021347
146         1.0     0.854954   -0.008756  0.008280
47          0.0     0.834182   -0.008756 -0.045336
109         1.0     0.796925   -0.008756 -0.008756
Model Bottom 5 Picks
 Avg rtn bottom 5: -0.0137
     true_value  model_value 

/home/mvkrein/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in long_scalars


Model Eval Results:
Training auc:  0.979864
Evaluation auc:  0.58882
Number of rounds:  307
Model Variable Importance:
                      fscore
w252_rank_p_L252_avg      55
w126_rank_p_L252_avg      30
w189_rank_p_L252_avg      27
w189_rank_p_L42_avg       26
w252_rank_p_L126_avg      26
w21_rank_p_L21_avg        25
w189_rank_p_L126_avg      24
w126_rank_p_L126_avg      24
w252_rank_p_L21_avg       24
w21_rank_p_L126_avg       24
*****************************
xgboost prediction - confusion matrix
 [[66  2]
 [82  3]] 
precision:  0.6
*****************************
Model Top 5 Picks
 Avg rtn top 5: 0.0037
     true_value  model_value  mkt_return   returns
146         1.0     0.985729   -0.002977  0.032005
120         0.0     0.933153   -0.002977 -0.006777
129         0.0     0.864004   -0.002977 -0.008199
13          1.0     0.839808   -0.002977  0.000385
60          1.0     0.835363   -0.002977  0.001160
Model Bottom 5 Picks
 Avg rtn bottom 5: -0.0101
     true_value  model_value  mk

Model Eval Results:
Training auc:  0.963632
Evaluation auc:  0.61126
Number of rounds:  141
Model Variable Importance:
                         fscore
w252_rank_p_L252_avg         37
w42_rank_p_L21_avg           27
w126_rank_p_L126_avg         27
w252_rank_p_L126_avg         22
w189_rank_p_L252_avg         21
w63_rank_p_L252_avg          20
w252_rank_p_L63_avg          20
ivv_w252_rank_p_L84_avg      18
w252_rank_p_L189_avg         18
w21_rank_p_L84_avg           17
*****************************
xgboost prediction - confusion matrix
 [[ 36   0]
 [112   5]] 
precision:  1.0
*****************************
Model Top 5 Picks
 Avg rtn top 5: 0.0054
     true_value  model_value  mkt_return   returns
71          1.0     0.913133   -0.022652  0.024448
107         1.0     0.860000   -0.022652 -0.010534
133         1.0     0.850515   -0.022652  0.028853
28          1.0     0.829864   -0.022652 -0.008831
18          1.0     0.802189   -0.022652 -0.006976
Model Bottom 5 Picks
 Avg rtn bottom 5: -0.

...

Model Eval Results:
Training auc:  0.692743
Evaluation auc:  0.531791
Number of rounds:  30
Model Variable Importance:
                         fscore
w252_rank_p_L252_avg         13
w252_rank_p_L21_avg           9
ivv_w21_rank_p_L252_avg       7
w189_rank_p_L189_avg          7
w10_rank_p_L63_avg            6
w84_rank_p_L10_avg            6
w63_rank_p_L05_avg            6
w189_rank_p_L42_avg           6
w189_rank_p_L252_avg          6
ivv_w189_rank_p_L42_avg       5
*****************************
xgboost prediction - confusion matrix
 [[106  10]
 [ 32   5]] 
precision:  0.333
*****************************
Model Top 5 Picks
 Avg rtn top 5: 0.0148
     true_value  model_value  mkt_return   returns
40          0.0     0.863215    0.016413 -0.011557
38          0.0     0.855075    0.016413  0.001299
28          0.0     0.845263    0.016413  0.004223
71          1.0     0.841300    0.016413  0.041820
151         1.0     0.831520    0.016413  0.037980
Model Bottom 5 Picks
 Avg rtn bottom 5: 0

Model Eval Results:
Training auc:  0.962104
Evaluation auc:  0.508371
Number of rounds:  137
Model Variable Importance:
                         fscore
w252_rank_p_L252_avg         42
w189_rank_p_L252_avg         30
w252_rank_p_L63_avg          29
w126_rank_p_L42_avg          24
w252_rank_p_L21_avg          20
w189_rank_p_L189_avg         20
w252_rank_p_L189_avg         18
w252_rank_p_L84_avg          16
w84_rank_p_L189_avg          16
ivv_w21_rank_p_L252_avg      16
*****************************
xgboost prediction - confusion matrix
 [[102  11]
 [ 25  15]] 
precision:  0.577
*****************************
Model Top 5 Picks
 Avg rtn top 5: 0.0656
    true_value  model_value  mkt_return   returns
16         1.0     0.966265    0.045146  0.055752
29         1.0     0.952607    0.045146  0.111626
28         1.0     0.951942    0.045146  0.091215
33         1.0     0.944798    0.045146  0.046301
14         0.0     0.943450    0.045146  0.023003
Model Bottom 5 Picks
 Avg rtn bottom 5: 0.0352

Model Eval Results:
Training auc:  0.930259
Evaluation auc:  0.491341
Number of rounds:  83
Model Variable Importance:
                      fscore
w252_rank_p_L252_avg      28
w189_rank_p_L252_avg      18
w252_rank_p_L63_avg       18
w126_rank_p_L42_avg       18
w252_rank_p_L10_avg       15
w252_rank_p_L42_avg       15
w189_rank_p_L189_avg      15
w42_rank_p_L42_avg        14
w84_rank_p_L189_avg       13
w126_rank_p_L63_avg       12
*****************************
xgboost prediction - confusion matrix
 [[122   8]
 [ 21   2]] 
precision:  0.2
*****************************
Model Top 5 Picks
 Avg rtn top 5: 0.0152
    true_value  model_value  mkt_return   returns
46         0.0     0.922964    0.034054  0.031648
23         1.0     0.921625    0.034054  0.071148
69         0.0     0.888158    0.034054 -0.037406
17         0.0     0.870925    0.034054  0.016393
11         0.0     0.869162    0.034054 -0.005698
Model Bottom 5 Picks
 Avg rtn bottom 5: 0.028
     true_value  model_value  mkt_re

Model Eval Results:
Training auc:  0.733619
Evaluation auc:  0.491552
Number of rounds:  31
Model Variable Importance:
                         fscore
w252_rank_p_L252_avg         10
w126_rank_p_L42_avg           9
w252_rank_p_L63_avg           8
w21_rank_p_L21_avg            8
w42_rank_p_L10_avg            7
w126_rank_p_L252_avg          7
ivv_w189_rank_p_L42_avg       7
w252_rank_p_L84_avg           7
w189_rank_p_L189_avg          7
w84_rank_p_L21_avg            6
*****************************
xgboost prediction - confusion matrix
 [[110  20]
 [ 20   3]] 
precision:  0.13
*****************************
Model Top 5 Picks
 Avg rtn top 5: -0.0231
    true_value  model_value  mkt_return   returns
50         0.0     0.949656    0.016351  0.014007
57         0.0     0.939881    0.016351 -0.048512
16         0.0     0.910568    0.016351 -0.019950
45         0.0     0.904908    0.016351 -0.019545
72         0.0     0.903106    0.016351 -0.041667
Model Bottom 5 Picks
 Avg rtn bottom 5: 0.015
 

Model Eval Results:
Training auc:  0.917193
Evaluation auc:  0.577807
Number of rounds:  73
Model Variable Importance:
                      fscore
w252_rank_p_L252_avg      26
w126_rank_p_L42_avg       18
w189_rank_p_L252_avg      15
w126_rank_p_L21_avg       14
w189_rank_p_L189_avg      14
w252_rank_p_L84_avg       14
w126_rank_p_L126_avg      12
w252_rank_p_L42_avg       12
w84_rank_p_L252_avg       10
w252_rank_p_L189_avg      10
*****************************
xgboost prediction - confusion matrix
 [[99 10]
 [41  3]] 
precision:  0.231
*****************************
Model Top 5 Picks
 Avg rtn top 5: -0.0033
     true_value  model_value  mkt_return   returns
97          0.0     0.925720     0.01647 -0.061425
52          0.0     0.899347     0.01647  0.012495
114         1.0     0.889997     0.01647  0.025706
147         0.0     0.866319     0.01647  0.009031
23          0.0     0.864678     0.01647 -0.002107
Model Bottom 5 Picks
 Avg rtn bottom 5: -0.0038
     true_value  model_value 